<a href="https://colab.research.google.com/github/nyanta012/open-model-tutorial/blob/main/section6/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabの使い方

- **セルの実行:** `Ctrl + Enter`, `Shift + Enter`
- **セルの作成:** `Ctrl + M B`
- **セルの削除:** `Ctrl + M D`

※GPUの設定が必要

# モデルの読み込み

In [1]:
%%capture
!pip install openai==0.28.1
!pip install transformers[torch]==4.34.1 accelerate==0.24.0 InstructorEmbedding==1.0.1 sentence_transformers==2.2.2 pypdf==3.16.4 optimum==1.13.2 auto-gptq==0.4.2 llama-index==0.9.8

from typing import Any, List

import torch
from InstructorEmbedding import INSTRUCTOR
from llama_index import ServiceContext, SimpleDirectoryReader, VectorStoreIndex
from llama_index.bridge.pydantic import PrivateAttr
from llama_index.embeddings.base import BaseEmbedding
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

In [2]:
model_name = "TheBloke/Xwin-LM-7B-V0.1-GPTQ"

In [3]:
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=2048,
    tokenizer_name=model_name,
    model_name=model_name,
    device_map="auto",
    model_kwargs={"torch_dtype": torch.float16},
    generate_kwargs={"temperature": 0.01, "do_sample": True},
)

# ベクトル化モデルの設定

In [4]:
class InstructorEmbeddings(BaseEmbedding):
    _model: INSTRUCTOR = PrivateAttr()
    _query_prefix: str = PrivateAttr()
    _text_prefix: str = PrivateAttr()

    def __init__(
        self,
        instructor_model_name: str = 'intfloat/multilingual-e5-large',
        **kwargs: Any,
    ) -> None:
        self._model = INSTRUCTOR(instructor_model_name)
        self._query_prefix = "query:" # multilingual-e5-largeの学習方法に準拠: https://huggingface.co/intfloat/multilingual-e5-large
        self._text_prefix = "passage:" # multilingual-e5-largeの学習方法に準拠: https://huggingface.co/intfloat/multilingual-e5-large
        super().__init__(**kwargs)

    @classmethod
    def class_name(cls) -> str:
        return "instructor"

    async def _aget_query_embedding(self, query: str) -> List[float]:
        return self._get_query_embedding(query)

    async def _aget_text_embedding(self, text: str) -> List[float]:
        return self._get_text_embedding(text)

    def _get_query_embedding(self, query: str) -> List[float]:
        embeddings = self._model.encode([[self._query_prefix, query]])
        return embeddings[0]

    def _get_text_embedding(self, text: str) -> List[float]:
        embeddings = self._model.encode([[self._text_prefix, text]])
        return embeddings[0]

    def _get_text_embeddings(self, texts: List[str]) -> List[List[float]]:
        embeddings = self._model.encode([[self._text_prefix, text] for text in texts])
        return embeddings

# LlamaIndexの設定

In [5]:
# ベクトル化モジュールの設定
service_context = ServiceContext.from_defaults(llm=llm,
    embed_model=InstructorEmbeddings(embed_batch_size=1), chunk_size=512
)
# ドキュメントをgithubからダウンロード
!mkdir -p data
!wget -P data https://github.com/nyanta012/open-model-tutorial/raw/main/pdf/健康のすべて.pdf

# ドキュメント読み込み
documents = SimpleDirectoryReader("data").load_data()

# インデックスの作成
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

load INSTRUCTOR_Transformer
max_seq_length  512
--2023-12-06 11:00:20--  https://github.com/nyanta012/open-model-tutorial/raw/main/pdf/%E5%81%A5%E5%BA%B7%E3%81%AE%E3%81%99%E3%81%B9%E3%81%A6.pdf
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/nyanta012/open-model-tutorial/main/pdf/%E5%81%A5%E5%BA%B7%E3%81%AE%E3%81%99%E3%81%B9%E3%81%A6.pdf [following]
--2023-12-06 11:00:20--  https://raw.githubusercontent.com/nyanta012/open-model-tutorial/main/pdf/%E5%81%A5%E5%BA%B7%E3%81%AE%E3%81%99%E3%81%B9%E3%81%A6.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 907748 (886K) [application/octet-stream]
Savi

# プロンプトの設定

In [6]:
TEMPLATE ="""A chat between a curious user and an artificial intelligence assistant.
The assistant gives helpful, detailed, and polite answers to the user's questions.
USER: 下記の情報が与えられています。
\n ---------------------\n {context_str} \n---------------------\n
この情報を参照して次の質問に答えてください。情報に基づいた回答のみ生成してください。
情報にない場合は、わからない旨を伝えてください。
質問:{query_str} ASSISTANT:"""

PROMPT = PromptTemplate(TEMPLATE)

query_engine = index.as_query_engine(text_qa_template=PROMPT, streaming=True)

# ドキュメントを参照したQ&Aの実行

In [7]:
response = query_engine.query("健康になるにはどんな運動をすると良いですか？")
response.print_response_stream()

健康になるには、適度な運動が最適です。適度な運動は、心血管の健康を促進し、筋力や柔軟性を向上させるだけでなく、ストレスの軽減や体重の管理にも役立ちます。また、有酸素運動（例：ウォーキング、ジョギング、サイクリング）、筋力トレーニング、柔軟性を高めるエクササイズなどをバランス良く取り入れることが重要です。

適度な運動を継続的に行うことで、健康な身体と心を維持し、生活の質を向上させることができます。ただし、個々の状況や目標に応じて、適切な運動を選択し、適切なレベルで行うことが重要です。また、日常生活でも積極的な動きを意識し、座っている時間を減らすことも健康促進につながります。</s>

健康になるには、適度な運動が最適です。適度な運動は、心血管の健康を促進し、筋力や柔軟性を向上させるだけでなく、ストレスの軽減や体重の管理にも役立ちます。また、有酸素運動（例：ウォーキング、ジョギング、サイクリング）、筋力トレーニング、柔軟性を高めるエクササイズなどをバランス良く取り入れることが重要です。

適度な運動を継続的に行うことで、健康な身体と心を維持し、生活の質を向上させることができます。ただし、個々の状況や目標に応じて、適切な運動を選択し、適切なレベルで行うことが重要です。また、日常生活でも積極的な動きを意識し、座っている時間を減らすことも健康促進につながります。</s>

In [14]:
print((response.source_nodes[0].node.get_text()))

適度な運動を行うことは、 心臓や血管の健康を促進し、
筋力や柔軟性を向上させるだけでなく、 ストレスの軽減や体重の管理にも役立ちます。 健康
な身体を維持するためには、有酸素運動（例：ウォーキング、ジョギング、サイクリング） 、
筋力トレーニング、柔軟性を高めるエクササイズなどをバランス良く取り入れることが重
要です。また、日常生活でも積極的な動きを意識し、座っている時間を減らすことも健康促
進につながります。 自身の体力や目標に合わせた適切な運動を継続的に行うこと で、 健康な
身体と心を維持し、生活の質を向上させることができます。  
 
２１．健康と環境への意識


In [15]:
print((response.source_nodes[1].node.get_text()))

食事は私たちの身体の燃料となり、成長や修復に
必要な栄養素を提供します。健康的な食事は、ビタミン、ミネラル、たんぱく質、繊維など
の栄養素をバランス良く摂取することを意味します。 また、 食事の多様性や適切なカロリー
摂取も重要です。健康な食事の基本は、野菜や果物、全粒穀物、健康な脂肪、良質なタンパ
ク質の摂取です。食事の栄養バランスに注意し、健康な身体と心をサポートしましょう。  
 
２０．健康と適度な運動の意義  
 
適度な運動は健康維持に不可欠です。 適度な運動を行うことは、 心臓や血管の健康を促進し、
筋力や柔軟性を向上させるだけでなく、 ストレスの軽減や体重の管理にも役立ちます。 健康
な身体を維持するためには、有酸素運動（例：ウォーキング、ジョギング、サイクリング） 、
筋力トレーニング、柔軟性を高めるエクササイズなどをバランス良く取り入れることが重
要です。
